<a href="https://colab.research.google.com/github/magnolia2001/Forest_Estimation/blob/main/notebooks/NNandCNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 窗口大小应为奇数，以保证标签在中间
size = 5 #define window size should be odd so that the label is in the middle
# 特征的形状，这里假设每个特征是一个大小为 (size, size) 的窗口，包含 90 个通道
shape = (90, size, size) #define shape of features
# np.ones(shape, dtype=None) 用于创建一个形状为 shape 的数组，并将所有元素初始化为 1.0
# 其中 shape：指定数组的形状，通常是一个整数或元组。 dtype：指定数组元素的数据类型（可选）。如果不指定，默认使用 float64 类型。
# labels1 用于存放标签数据（掩膜）, data1 用于存放提取的特征数据
# np.ones(1)返回的是一个只有一个元素的数组，其中该元素值为 1。
labels1 = np.ones(1) #array for labels
# 创建了一个数组，形状为 shape 即 (90, 5, 5) 的 NumPy 数组，并且所有的元素值都被初始化为 1.0 。
data1 = np.ones(shape) #array for features
# 扩展维度，便于后续拼接操作
data1 = np.expand_dims(data1, axis=0) #expand dimension to concatenate

# 遍历目录中的图像（假设有 20 张图像, 具体数量还需要根据自己的情况修改）
# 在 for j in range(20) 这个遍历过程中，区分 j < 10 和 j >= 10 的目的是为了处理不同的文件命名规则。
# 对于小于 10 的文件名，文件名是 "image_00X.npy"，其中 X 是单个数字（0 到 9）。
# 对于大于等于 10 的文件名，文件名是 "image_0XY.npy"，其中 XY 是两位数的数字（10 到 19）。
for j in range(20): #iterate over images in directory
  if j < 10:
    # 读取图像数据
    X = np.load('/content/images/image_00'+ str(j) + '.npy')
    # 读取掩膜数据
    y = np.load('/content/masks/mask_00'+ str(j) + '.npy')
    # 选择掩膜图像 y 中所有大于 0 的位置（即标签不为 0 的位置），并返回这些位置的索引。
    # y > 0 是一个布尔条件，返回一个与 y 相同形状的布尔数组, 如果是大于 0，布尔值为 True，否则为 False。
    # np.argwhere() 是 NumPy 库中的一个函数，它返回数组中满足某个条件的所有索引（行列坐标），即满足条件的元素的坐标位置。
    # 由于 indices 是一个二维数组，形状通常是 (N, 2)，其中 N 是符合条件的元素的个数，2 表示每个位置的二维索引（x 和 y 坐标）。
    indices = np.argwhere(y > 0) #select all values with label
    # 提取二维索引的 y 坐标
    # 切片操作 indices[:, 1:] 就是提取所有行中的第二列（即 x 和 y 坐标中的 y 坐标）。
    indices_2d = indices[:, 1:] #extract indices
    # 创建一个存储索引的数组
    # np.ones(2) 会创建一个包含 2 个元素的数组，所有元素的值为 1. 。
    # .reshape(-1, 2) 将该数组的形状重塑为 (-1, 2)，表示按列数为 2 进行重塑，-1 表示自动计算行数。由于只有 2 个元素，这会将数组变成形状为 (1, 2) 的二维数组。
    ind_y = np.ones(2).reshape(-1,2) #array to collect indices

    # 遍历掩膜中的每个标签位置
    for i in indices_2d: #iterate over indices
      if shape == X[:, i[0] - (size//2):i[0] + (size//2) + 1, i[1] - (size//2):i[1] + (size//2) + 1].shape: #select only features with the same shape because of labels at the image border
        # 提取图像窗口
        # 确保当前窗口大小与指定的窗口大小一致
        temp = X[:, i[0] - (size//2):i[0] + (size//2) + 1, i[1] - (size//2):i[1] + (size//2) + 1] #save them temporary
        # 扩展维度以便拼接
        temp2 = np.expand_dims(temp, axis=0) #expand dimension to concatenate
        # 拼接特征数据
        data1 = np.concatenate((data1, temp2), axis=0) #concatenation
        # 拼接标签索引
        ind_y = np.concatenate((ind_y, i.reshape(-1,2)), axis=0) #concatenation of index so that they have the same order and length as the features

    # 去掉第一个虚拟值
    ind_y = ind_y[1:] #remove first dummy values
    # 提取 x 坐标
    indices_1 = ind_y[:, 0].astype(int)
    # 提取 y 坐标
    indices_2 = ind_y[:, 1].astype(int)
    # 提取标签值
    data_y = y[0, indices_1, indices_2] #extract labels
    # 拼接标签
    labels1 = np.concatenate((labels1, data_y), axis = 0) #concatenate labels

  if j >= 10:
    X = np.load('/content/images/image_0'+ str(j) + '.npy')
    y = np.load('/content/masks/mask_0'+ str(j) + '.npy')
    indices = np.argwhere(y > 0)
    indices_2d = indices[:, 1:]
    ind_y = np.ones(2).reshape(-1,2)
    for i in indices_2d:
      if shape == X[:, i[0] - (size//2):i[0] + (size//2) + 1, i[1] - (size//2):i[1] + (size//2) + 1].shape:
        temp = X[:, i[0] - (size//2):i[0] + (size//2) + 1, i[1] - (size//2):i[1] + (size//2) + 1]
        temp2 = np.expand_dims(temp, axis=0)
        data1 = np.concatenate((data1, temp2), axis=0)

        ind_y = np.concatenate((ind_y, i.reshape(-1,2)), axis=0)

    ind_y = ind_y[1:]
    indices_1 = ind_y[:, 0].astype(int)
    indices_2 = ind_y[:, 1].astype(int)
    data_y = y[0, indices_1, indices_2]
    labels1 = np.concatenate((labels1, data_y), axis = 0)

data1 = data1[1:] #remove first dummy values
labels1 = labels1[1:] #remove first dummy values